<a href="https://colab.research.google.com/github/MuhammadUsaid/Qutron_AI/blob/master/Qutron1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install folium==0.2.1
!pip install qiskit

In [ ]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import Aer, execute
from qiskit.tools.visualization import plot_histogram, plot_state_city
import random
Aer.backends()

from qiskit.providers.aer import QasmSimulator, StatevectorSimulator, UnitarySimulator

def binary(number, n):
  '''The function to convert the number into binary of n bits'''
  number = bin(number)
  number = number[2:]
  n = n - len(number)
  for i in range(n):
    number = '0' + number
  return number

def seqOfNotGates(idx, n):
  '''The function to find the number of X gates for the implementation of sign flip blocks'''
  idx = binary(idx, n)
  seq = []
  for i in range(len(idx)):
    if int(idx[i]) != 1:
      seq.append(i)
  return seq
def prepareWiState(inputs, n):
  '''The function to implement Ui transformation
    Takes inputs which is a vector and n is the number of qubits'''
  for idx in range(len(inputs)): #Input is the vector containing 1 & -1 inputs
    if inputs[idx] == -1:
      sequence = seqOfNotGates(idx, n)
      for i in sequence:
        circ.x(int(i))
      circ.cz(0,1)
      for i in sequence:
        circ.x(int(i))
def prepareWwState(inputs, n):
  '''The function to implement Uw transformation'''
  prepareWiState(inputs, n)
  for i in range(n):
    circ.h(i)
  for i in range(n):
    circ.x(i)
#i is the input
#w is the weight
i = [1, 1, 1, 1]
inputVectors = [
      [1,1,1,1], [1,-1,1,-1], [1,1,1,-1], [1,1,-1,-1], [-1,-1,-1,-1], [1,1,-1,1], [-1,-1,1,1], [1,1,1,1], [-1,1,1,-1], [-1,1,1,1]
]
actualWeights = [-1,-1,-1,-1]
labels = [1,1,1,1,1,1,1,0,1,0]
circ = QuantumCircuit(3,1)
def quantumPerceptron(inputs, weights, n):
  '''Function to implement Actual Quantum perceptron'''
  for i in range(n):
    circ.h(i) #Implementing Hadamard Gates to make superposition state
  prepareWiState(inputs, n)
  prepareWwState(weights, n)
  circ.ccx(0,1,2)
  circ.measure(2,0)
def chooseRandomIndex(indexes):
  chosen = []
  changeIndexes = []
  for i in range(len(indexes)//2):
    r = random.randint(0,len(indexes)-1)
    #print("R", r)
    while r in chosen:
      r = random.randint(0,len(indexes)-1)
    chosen.append(r)
    changeIndexes.append(indexes[r])
  return changeIndexes
def changeWeights(weights, inputs, close):
  sameIndexes = []
  diffIndexes = []
  newWeights = weights.copy()
  #print("Change W")
  for i in range(len(weights)):
    if weights[i] != inputs[i]:
      diffIndexes.append(i)
    else:
      sameIndexes.append(i)
  #print("I",inputs)
  #print("W", weights)
  #print("Same Index", sameIndexes)
  #print("Diff Index", diffIndexes)
  if close:
    changeIndexes = chooseRandomIndex(diffIndexes)
    for i in changeIndexes:
      newWeights[i] = inputs[i]
  else:
    changeIndexes = chooseRandomIndex(sameIndexes)
    for i in changeIndexes:
      newWeights[i] = -1 * inputs[i]
  #print("W2", newWeights)
  return newWeights

def training(inputVectors, labels):
  results = []
  weights = [1,1,1,1]
  for i in range(len(inputVectors)):
    quantumPerceptron(inputVectors[i], weights, 2)
    circ.draw(output = 'mpl')
    simulator = Aer.get_backend('qasm_simulator')
    result = execute(circ, simulator).result()
    counts = result.get_counts(circ)
    if '1' in counts:
      prob = counts['1']/1024
    else:
      prob = 0
    #print(counts)
    print(i, prob)
    if prob > 0.8:
      if labels[i] != 1:
        weights = changeWeights(weights, inputVectors[i], True)
    else:
      if labels[i] != 0:
        weights = changeWeights(weights, inputVectors[i], False)
    print(weights)
training(inputVectors, labels)

0 1.0
[1, 1, 1, 1]
1 1.0
[1, 1, 1, 1]
2 0.740234375
[1, 1, -1, 1]
3 0.7568359375
[-1, 1, -1, 1]
4 0.265625
[1, 1, -1, 1]
5 0.236328125
[1, -1, 1, 1]
6 0.4033203125
[1, -1, 1, -1]
7 0.4951171875
[1, -1, 1, -1]
8 0.609375
[1, -1, -1, -1]
9 0.490234375
[1, -1, -1, -1]


In [ ]:
'''Code to generate labels'''
weights = [-1, -1, -1, -1]
inputVectors = [
      [1,1,1,1], [1,-1,1,-1], [1,1,1,-1], [1,1,-1,-1], [-1,-1,-1,-1], [1,1,-1,1], [-1,-1,1,1], [1,1,1,1], [-1,1,1,-1], [-1,1,1,1], [1,1,1,1],[1,-1,-1,-1], [1,1,1,-1],
      [1,-1,-1,1], [-1,-1,1,-1]
]
for i in inputVectors:
  quantumPerceptron(i, weights, 2)
  circ.draw(output = 'mpl')
  simulator = Aer.get_backend('qasm_simulator')
  result = execute(circ, simulator).result()
  counts = result.get_counts(circ)
  print(counts)

{'0': 665, '1': 359}
{'0': 512, '1': 512}
{'0': 511, '1': 513}
{'0': 527, '1': 497}
{'0': 523, '1': 501}
{'0': 510, '1': 514}
{'0': 511, '1': 513}
{'0': 481, '1': 543}
{'0': 496, '1': 528}
{'0': 519, '1': 505}
{'0': 505, '1': 519}
{'0': 517, '1': 507}
{'0': 501, '1': 523}
{'0': 491, '1': 533}
{'0': 523, '1': 501}
